In [5]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path

# raw text to list

In [4]:
Path.cwd()

PosixPath('/Users/smap10/Project/aspect-extraction/notebooks')

In [8]:
Path.cwd().parent

PosixPath('/Users/smap10/Project/aspect-extraction')

In [11]:
train_file = Path.cwd().parent.joinpath('raw-data/semeval-2016/train.xml')
train_file

PosixPath('/Users/smap10/Project/aspect-extraction/raw-data/semeval-2016/train.xml')

In [34]:
soup = None
with train_file.open(encoding="utf-8") as f:
    soup = BeautifulSoup(f.read().strip(), "lxml-xml")

if soup is None:
    raise Exception("Can't read xml file")

sentence_nodes = soup.find_all("sentence")

In [41]:
len(sentence_nodes)

2000

In [35]:
for i, node in enumerate(sentence_nodes):
    print(node)
    print()
    if i > 5:
        break

<sentence id="1004293:0">
<text>Judging from previous posts this used to be a good place, but not any longer.</text>
<Opinions>
<Opinion category="RESTAURANT#GENERAL" from="51" polarity="negative" target="place" to="56"/>
</Opinions>
</sentence>

<sentence id="1004293:1">
<text>We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.</text>
<Opinions>
<Opinion category="SERVICE#GENERAL" from="75" polarity="negative" target="staff" to="80"/>
</Opinions>
</sentence>

<sentence id="1004293:2">
<text>They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.</text>
<Opinions>
<Opinion category="SERVICE#GENERAL" from="0" polarity="negative" target="NULL" to="0"/>
</Opinions>
</sentence>

<sentence id="1004293:3">
<text>The food was lousy - too sweet or too salty and the portions tiny.</text>
<Opinions>
<Opinion category="FOOD#QUALITY" from="4" polari

In [90]:
# some sentence only contain text tag, no Opinions tag
i = 0
for n in sentence_nodes:
    if not n.find('Opinions'):
        print(n)    
        i += 1
        if i > 3:
            break

<sentence id="1041457:0">
<text>I had my eyes on this place, promising myself I will one day 'give it a try.' </text>
</sentence>
<sentence OutOfScope="TRUE" id="1041457:5">
<text>Anyways, if you're in the neighborhood to eat good food, I wouldn't waste my time trying to find something, rather go across the street to Tamari.</text>
</sentence>
<sentence id="1053543:0">
<text>We ate outside at Haru's Sake bar because Haru's restaurant next door was overflowing.</text>
</sentence>
<sentence id="1053543:1">
<text>What's the difference between the two?</text>
</sentence>


In [105]:
sentences = []
i = 0
for n in sentence_nodes:
    i += 1
    sentence = {}
    aspect_term = []
    sentence['id'] = i
    sentence['text'] = n.find('text').string
    if n.find('Opinions'):
        for c in n.find('Opinions').contents:
            if c.name == 'Opinion':
                if c['target'] not in aspect_term:
                    aspect_term.append(c['target'])

    sentence['aspect'] = aspect_term
    sentences.append(sentence)

In [106]:
sentences[:5]

[{'aspect': ['place'],
  'id': 1,
  'text': 'Judging from previous posts this used to be a good place, but not any longer.'},
 {'aspect': ['staff'],
  'id': 2,
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.'},
 {'aspect': ['NULL'],
  'id': 3,
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.'},
 {'aspect': ['food', 'portions'],
  'id': 4,
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.'},
 {'aspect': ['NULL'],
  'id': 5,
  'text': 'After all that, they complained to me about the small tip.'}]

In [107]:
len([])

0

In [108]:
# No sentence contain two NULL target (aspect term)
aspect_zero = 0
aspect_one = 0

for s in sentences:
    # aspect == [] or aspect == ['NULL']
    if len(s['aspect'])==0 or s['aspect'][0] == 'NULL':
        aspect_zero += 1
    else:
        aspect_one += 1
        
print(len(sentences)) # Total 2000 sentences
print(aspect_zero) # xxx sentences have no aspect term
print(aspect_one) # xxx sentences have at least 1 aspect term

2000
791
1209


In [ ]:
# print aspect and text 
i = 0
for s in sentences:
    if len(s['aspect']) > 1:
        i += 1
        print(s['aspect'])
        print(s['text'])
print(i)

In [112]:
sentences[:20]

[{'aspect': ['place'],
  'id': 1,
  'text': 'Judging from previous posts this used to be a good place, but not any longer.'},
 {'aspect': ['staff'],
  'id': 2,
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.'},
 {'aspect': ['NULL'],
  'id': 3,
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.'},
 {'aspect': ['food', 'portions'],
  'id': 4,
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.'},
 {'aspect': ['NULL'],
  'id': 5,
  'text': 'After all that, they complained to me about the small tip.'},
 {'aspect': ['place'], 'id': 6, 'text': 'Avoid this place!'},
 {'aspect': ['food'],
  'id': 7,
  'text': 'I have eaten at Saul, many times, the food is always consistently, outrageously good.'},
 {'aspect': ['Saul'],
  'id': 8,
  'text': 'Saul is the best restaurant on Smith Street and i

##  raw text to list - sum it up
We have total 2000 sentences, and only 1209 sentences have 1 aspect term at least. Sum above to one function.

In [113]:
def raw2dict(sentence_nodes):
    """
    Input: a soup object, e.g. soup.find_all("sentence")
    Output: a list of dictionaries, contains id, text, aspect terms
    """
    sentences = []
    i = 0
    for n in sentence_nodes:
        i += 1
        sentence = {}
        aspect_term = []
        sentence['id'] = i
        sentence['text'] = n.find('text').string
        if n.find('Opinions'):
            for c in n.find('Opinions').contents:
                if c.name == 'Opinion':
                    if c['target'] not in aspect_term:
                        aspect_term.append(c['target'])

        sentence['aspect'] = aspect_term
        sentences.append(sentence)
        
    return sentences

In [114]:
train_file = Path.cwd().parent.joinpath('raw-data/semeval-2016/train.xml')
soup = None
with train_file.open(encoding="utf-8") as f:
    soup = BeautifulSoup(f.read().strip(), "lxml-xml")

if soup is None:
    raise Exception("Can't read xml file")

sentence_nodes = soup.find_all("sentence")
sentences = raw2dict(sentence_nodes)

In [116]:
sentences[:5]

[{'aspect': ['place'],
  'id': 1,
  'text': 'Judging from previous posts this used to be a good place, but not any longer.'},
 {'aspect': ['staff'],
  'id': 2,
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.'},
 {'aspect': ['NULL'],
  'id': 3,
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.'},
 {'aspect': ['food', 'portions'],
  'id': 4,
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.'},
 {'aspect': ['NULL'],
  'id': 5,
  'text': 'After all that, they complained to me about the small tip.'}]

# list to dataframe

the dataframe should have 3 columns, sentence id, word, tag


In [117]:
test = sentences[:5]
test

[{'aspect': ['place'],
  'id': 1,
  'text': 'Judging from previous posts this used to be a good place, but not any longer.'},
 {'aspect': ['staff'],
  'id': 2,
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.'},
 {'aspect': ['NULL'],
  'id': 3,
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.'},
 {'aspect': ['food', 'portions'],
  'id': 4,
  'text': 'The food was lousy - too sweet or too salty and the portions tiny.'},
 {'aspect': ['NULL'],
  'id': 5,
  'text': 'After all that, they complained to me about the small tip.'}]

In [166]:
# split sentences to words, and contain the punctuations
import re
s = sentences[0]['text']
print(s)

s = re.sub('([.,!?()])', r' \1 ', s) # match the punctuation characters and surround them by spaces,
s = re.sub('\s{2,}', ' ', s)         # collapse multiple spaces to one space
print(s.lower().split())

Judging from previous posts this used to be a good place, but not any longer.
['judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.']


#### a example to assign IOB format 

In [136]:
s = ['judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.']
aspects = ['a good place', 'Posts']

aspects = [x.lower() for x in aspect]
aspects

['a good place', 'posts']

In [145]:
tags = ['O'] * len(s)
print(tags)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [156]:
tags = ['O'] * len(s)
print(tags)

for aspect in aspects:
    pre_index = 0
    for word in s: 
        if word in aspect: # 'good' in 'a good place'
            print(word)
            cur_index = s.index(word) # 
            print(cur_index, pre_index)
            if cur_index - pre_index == 1: # good after a
                tags[cur_index] = 'I'
#                 print('I')
            else:                       # a after no aspect words
                tags[cur_index] = 'B'
            pre_index = cur_index 
            
print(tags)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
a
8 0
good
9 8
place
10 9
posts
3 0
['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O']


In [174]:
def tagging_IOB(s, aspects):
    """Assigning IOB tag to each word in s
    Input: 
        s: sentences, a list of words, e.g. ['judging', 'from', 'previous', 'posts']
        aspects: a list of aspect term, e.g. ['a good place', 'Posts']
    Output:
        tag: a list of tag, e.g. ['O', 'O', 'O', 'B']
    """
    tags = ['O'] * len(s)

    for aspect in aspects:
        pre_index = 0
        for word in s: 
            if word in aspect: # 'good' in 'a good place'
                cur_index = s.index(word) 
                if cur_index - pre_index == 1: # inside an aspect term
                    tags[cur_index] = 'I'
                else:                       # beginning of an aspect term
                    tags[cur_index] = 'B'
                pre_index = cur_index 
    return tags


#### a example to assign IOB format ----end------

In [175]:
import re

def split2words(s_text):
    """Split string with white and prereserve the punctuation
    Input:
        s_text: string, a sentence, e.g. Judging from previous posts this used to be a good place, but not any longer.
    Output:
        words: a list of words, e.g. ['judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.']
    """
    s_text = re.sub('([.,!?()])', r' \1 ', s_text) # match the punctuation characters and surround them by spaces,
    s_text = re.sub('\s{2,}', ' ', s_text)         # collapse multiple spaces to one space
    words = s_text.lower().split()
    return words

In [190]:
import re

sentences_df = []
for s in sentences:
    sentence = {}
    sentence['Sentence #'] = s['id']
    sentence['Word'] = split2words(s['text']) 
    s_length = len(sentence['Word']) # the length of sentence, used to generate tag
    if len(s['aspect'])==0 or s['aspect'][0] == 'NULL':
        sentence['Tag'] = ['O'] * s_length
    else:
        aspect_terms = [x.lower() for x in s['aspect']]  # IOB format if aspect exist
        sentence['Tag'] = tagging_IOB(sentence['Word'], aspect_terms)
    
    sentences_df.append(sentence)


# list to dataframe - sum it up

In [ ]:
def list_of_dic(sentences):
    """Convert list of dict to dataframe
    Input: 
        sentences: a list of dictionaries, contains id, text, aspect terms. The output of raw2dict
    Output:
        data: a dataframe with three columns, sentence id, words, tag with IOB format
    """
    data = pd.DataFrame()
    for s in sentences:
        sentence = {}
        sentence['Sentence #'] = s['id']
        sentence['Word'] = split2words(s['text'])  # split text to words
        s_length = len(sentence['Word']) # the length of sentence, used to generate tag
        if len(s['aspect'])==0 or s['aspect'][0] == 'NULL': # tagging: if no aspect term
            sentence['Tag'] = ['O'] * s_length
        else:                                               # IOB format tag if aspect exist
            aspect_terms = [x.lower() for x in s['aspect']]  
            sentence['Tag'] = tagging_IOB(sentence['Word'], aspect_terms)

        # convert each setence to dataframe 
        sentence_df = pd.DataFrame.from_dict(sentence)
        data = data.append(sentence_df, ignore_index=True)
    
    return data

In [194]:
# for s in sentences_df:
#     s_id = s['Sentence #']
#     s['Sentence #'] = [s_id] * len(s['Tag'])
    
# sentences_df[:2]

In [191]:
sentences_df[1989]

{'Sentence #': 1990,
 'Tag': ['O', 'O', 'O', 'O', 'B', 'O'],
 'Word': ['i', 'generally', 'like', 'this', 'place', '.']}

In [193]:
pd.DataFrame.from_dict(sentences_df[0])

,Sentence #,Tag,Word
0,1,O,judging
1,1,O,from
2,1,O,previous
3,1,O,posts
4,1,O,this
5,1,O,used
6,1,O,to
7,1,O,be
8,1,B,a
9,1,O,good


In [198]:
data = pd.DataFrame()
for s in sentences_df:
    s_df = pd.DataFrame.from_dict(s)
    data = data.append(s_df, ignore_index=True)


In [199]:
data

,Sentence #,Tag,Word
0,1,O,judging
1,1,O,from
2,1,O,previous
3,1,O,posts
4,1,O,this
5,1,O,used
6,1,O,to
7,1,O,be
8,1,B,a
9,1,O,good


In [200]:
save_file = Path.cwd().parent.joinpath('data/semeval-2016/train.csv')

data.to_csv(save_file, index=False)